In [1]:
import dpkt
import matplotlib.pyplot as plt
import numpy
import math
# import pickle5 as pickle
import pickle
import numpy as np
import matplotlib
import os
import time
from datetime import datetime
import json
import csv
from pathlib import Path

In [2]:
TransitionPeriod = 4
amazon_ips = {}
GAMMA = 0.25
observation_lengths = [180, 300, 900, 1800] # in seconds
LENGTH = 3600

In [ ]:
def get_message_file_names(b, e, messages_dir):
    all_messages_path = 'path/to/generated_messages/messages_pareto_xm_5000_alpha_0.93_max30delay_ver02'
    all_messages_file_names = sorted(os.listdir(all_messages_path))
    if not os.path.exists(messages_dir):
        os.makedirs(messages_dir)
    for i in range(len(all_messages_file_names)):
        if i >= b and i <= e:
            print(all_messages_file_names[i])
            os.system('cp {} {}'.format(os.path.join(all_messages_path, all_messages_file_names[i]), os.path.join(messages_dir,all_messages_file_names[i])))
    return


root_dir = 'path/to/root/directory/of/traces'


timestamps_dirs = []
messages_dirs = []
message_file_names = []
packets_pickle_dirs = []
packets = []
dir_count = 0
for path in sorted(Path(root_dir).iterdir()):
    if path.is_dir() and path.name != 'aggregate' and not path.name.endswith('extra'):
        print(path)
        timestamps_dirs.append(os.path.join(path, 'timestamps'))
        print('timestamps are in {}'.format(timestamps_dirs[-1]))
        messages_dir = Path(os.path.join(path, 'messages'))
        messages_dir.mkdir(parents=True, exist_ok=True)
        messages_dirs.append(os.path.join(path, 'messages'))
        print('messages are in {}'.format(messages_dirs[-1]))
        range_begin = int(path.name.split('_')[0])
        range_end = int(path.name.split('_')[1])
        print(range_begin, range_end)
        get_message_file_names(range_begin, range_end, messages_dirs[dir_count])
        message_file_names.extend(sorted(os.listdir(messages_dirs[dir_count])))
        print('length of message_file_names is {}'.format(len(message_file_names)))
        packets_pickle_dirs.append(os.path.join(path, 'traces/pickles/packets'))
        # read all packets from packets pickle and write them to 'packets'
        pickle_name = os.listdir(packets_pickle_dirs[dir_count])[0]
        print('loading {} ...'.format(pickle_name))
        start = time.time()
        with open(os.path.join(packets_pickle_dirs[dir_count], pickle_name[:-7] + '.pickle'), 'rb') as handle:
            packets.extend(pickle.load(handle))
        end = time.time()
        hours, rem = divmod(end - start, 360)
        mins, seconds = divmod(rem, 60)
        print('loading took {:0>2}:{:0>2}:{:05.2f}'.format(int(hours), int (mins), seconds))
        dir_count += 1
aggregate_dir = Path(root_dir) / 'aggregate'
aggregate_dir.mkdir(parents=True, exist_ok=True)
results_dir = str(aggregate_dir)

In [26]:
def get_sizes_sender(messages_file, max_count=None):
    sizes = []
    with open(messages_file) as f:
        data = json.load(f)
        count = 0
        for message in data['messages']:
            if max_count is None or count < max_count:
                sizes.append(int(message['size'])) # sizes are in bytes
            else:
                break
            count += 1
    return sizes

def get_timestamps_sender(timestamps_file, max_count=None):
    timestamps = []
    with open(timestamps_file) as f:
        lines = f.readlines()
        count = 0
        for line in lines:
            if max_count is None or count < max_count:
                line = line.split()
                timestamps.append((float)(line[2]))
            else:
                break
            count += 1
    return timestamps

def get_sizes_of_all_senders(messages_dir):
    message_file_names = sorted(os.listdir(messages_dir))
    all_message_sizes_of_senders = []
    for mf_name in message_file_names:
        message_sizes = get_sizes_sender(os.path.join(messages_dir, mf_name))
        all_message_sizes_of_senders.append(message_sizes)
    return all_message_sizes_of_senders

def get_timestamps_of_all_senders(messages_dir, timestamps_root_dir):
    message_file_names = sorted(os.listdir(messages_dir))
    all_timestamps = []
    for mf_name in message_file_names:
        timestamps = get_timestamps_sender(os.path.join(timestamps_root_dir, 'timestamps_' + mf_name[:-5] + '.txt'))
        all_timestamps.append(timestamps)
    return all_timestamps

def get_types_of_all_senders(messages_dir, max_count=None):
    message_file_names = sorted(os.listdir(messages_dir))
    all_types = []
    for messages_file in message_file_names:
        with open(os.path.join(messages_dir, messages_file)) as f:
            types = []
            data = json.load(f)
            count = 0
            for message in data['messages']:
                if max_count is None or count < max_count:
                    types.append((message['type']))
                else:
                    break
                count += 1
            all_types.append(types)
    return all_types

In [ ]:
timestamps_of_all_senders = []
message_sizes_of_all_senders = []
message_types_of_all_senders = []
for i in range(dir_count):
    timestamps_of_all_senders.extend(get_timestamps_of_all_senders(messages_dirs[i], timestamps_dirs[i]))
    message_sizes_of_all_senders.extend(get_sizes_of_all_senders(messages_dirs[i]))
    message_types_of_all_senders.extend(get_types_of_all_senders(messages_dirs[i]))

In [32]:
packets[500]

{'to_amazon': False,
 'src': '192.168.122.91',
 'dst': '192.168.122.75',
 'size': 64,
 'timestamp': 1634137925.66455,
 'protocol': 6,
 's_port': 1080,
 'd_port': 40940}

In [33]:
def separate_receiver_packets_non_chronological_traces(packets, timestamps_of_all_senders):
    packets_of_all_receivers = []
    all_beginnings = []
    all_endings = []
    for sender_timestamps in timestamps_of_all_senders:
        beginning = sender_timestamps[0] - 5 #seconds
        ending = sender_timestamps[len(sender_timestamps)-1] + 30 #seconds
        all_beginnings.append(beginning)
        all_endings.append(ending)
    print(all_beginnings)
    print(all_endings)
    current_sender = 0
    num_of_senders = len(timestamps_of_all_senders)
    current_sender_received_packets = []
    flag = 0
    for p in packets:
        if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
            current_sender_received_packets.append(p)

        elif current_sender == num_of_senders - 1:
            packets_of_all_receivers.append(current_sender_received_packets)
            break
        elif p.get('timestamp') >= all_beginnings[current_sender + 1] and p.get('timestamp') < all_endings[current_sender + 1]:
            packets_of_all_receivers.append(current_sender_received_packets)
            current_sender += 1
            current_sender_received_packets = []
            current_sender_received_packets.append(p)
    return packets_of_all_receivers

def separate_receiver_packets_chronological_traces(packets, timestamps_of_all_senders):
    packets_of_all_receivers = []
    all_beginnings = []
    all_endings = []
    for sender_timestamps in timestamps_of_all_senders:
        beginning = sender_timestamps[0] - 5 #seconds
        ending = sender_timestamps[len(sender_timestamps)-1] + 30 #seconds
        all_beginnings.append(beginning)
        all_endings.append(ending)
    print(all_beginnings)
    print(all_endings)
    current_sender = 0
    num_of_senders = len(timestamps_of_all_senders)
    current_sender_received_packets = []
    for p in packets:            
        if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
            current_sender_received_packets.append(p)
        elif p.get('timestamp') >= all_endings[current_sender]:
            packets_of_all_receivers.append(current_sender_received_packets)
            if current_sender < num_of_senders - 1:
                current_sender += 1
                current_sender_received_packets = []
                if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
                    current_sender_received_packets.append(p)
            else:
                break
    return packets_of_all_receivers

In [ ]:
packets_of_all_receivers = separate_receiver_packets_non_chronological_traces(packets, timestamps_of_all_senders)
packets_towards_receivers = [[p for p in packets if (p.get('to_amazon') is False)] for packets in packets_of_all_receivers]

In [37]:
def convert_time(timestamps, start_time):
    converted_timestamps = []
    for t in timestamps:
        converted = t - start_time
        if converted < 0:
            print('Error: timestamp is larger than start time with t = {} and start_time = {}'.format(t, start_time))
        converted_timestamps.append(converted)
    return converted_timestamps    

def get_on_periods_of_sender_2(timestamps, message_sizes):
    global TransitionPeriod
    bursts = []
    for t, s in zip(timestamps, message_sizes):
        bursts.append((t, s))
    return bursts

def get_on_periods_of_user(timestamps, user_packet_sizes):
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    for time_iter in range(0, len(timestamps)):
        if time_iter == len(timestamps) - 1 and is_in_burst == True:
            if burst_size > 1514:
                bursts.append((timestamps[last_burst_index], burst_size))
                break
        if timestamps[time_iter] - timestamps[last_burst_index] > TransitionPeriod:
            if is_in_burst == True:
                if burst_size > 1514:
                    bursts.append((timestamps[last_burst_index], burst_size))
                is_in_burst = False
                burst_size = 0
        if user_packet_sizes[time_iter] > 1400:
            if is_in_burst == False:
                is_in_burst = True
                last_burst_index = time_iter
            burst_size += user_packet_sizes[time_iter]
    return bursts

def get_bursts_user(timestamps, user_packet_sizes):
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    for time_iter in range(0, len(timestamps)):
        if time_iter == len(timestamps) - 1 and is_in_burst == True:
            if burst_size > 1500:
                bursts.append((timestamps[last_burst_index], burst_size))
                break
        if timestamps[time_iter] - timestamps[last_burst_index] > 1:
            if is_in_burst == True:
                if burst_size > 1500:
                    bursts.append((timestamps[time_iter], burst_size))
                is_in_burst = False
                burst_size = 0
        if user_packet_sizes[time_iter] > 40:
            if is_in_burst == False:
                is_in_burst = True
            last_burst_index = time_iter
            burst_size += user_packet_sizes[time_iter]
    return bursts

def get_bursts_user_2(timestamps, user_packet_sizes):
    bursts = []
    burst_size = 0
    for time_iter in range(0, len(timestamps) - 1):
        ipd = timestamps[time_iter + 1] - timestamps[time_iter]
        if ipd < 1:
            if user_packet_sizes[time_iter] > 40:
                burst_size += user_packet_size[time_iter]
        else:
            if burst_size > 20000:
                bursts.append((timestamps[time_iter], burst_size))
            burst_size = 0
    return bursts

def merge_channel_period_points(ChannelPeriodPoints):
    c = []
    time = [x[0] for x in ChannelPeriodPoints]
    size = [x[1] for x in ChannelPeriodPoints]
    burst_time = []
    burst_volume = []
    burst_volume.append(size[0])
    burst_time.append(time[0])
    temp = time[0]
    for i in range(len(time)-1):
        if time[i+1] - temp < 2:
            burst_volume[-1] += size[i+1]
            temp = time[i+1]
        else:
            burst_time.append(time[i+1])
            burst_volume.append(size[i+1])
            temp = time[i+1]
    for i in range(len(burst_time)):
        c.append((burst_time[i], burst_volume[i]))
    return c

def get_channel_points(f):
    ChannelPeriodPoints = []
    first_line = True
    for line in f:
        line = line.split()
        if first_line:
            find_start_time(line[1].decode('ascii'))
            first_line = False
            continue
        message_type = line[4].decode('ascii')
        if message_type == 'text':
            continue
        time = convert_time(line[2].decode('ascii'))
        if line[5].decode('ascii') == 'None':
            size = 300000
        else:
            size = int(line[5].decode('ascii'))
        ChannelPeriodPoints.append((time, size))
    if len(ChannelPeriodPoints) == 0:
        return []
    c = merge_channel_period_points(ChannelPeriodPoints)
    return c

In [ ]:
all_user_bursts = []
all_channel_bursts = [] # the same as adversary bursts in the case of one-on-one chats

for i in range(0, len(packets_towards_receivers)):

    print('Hour {} {}'.format(i, message_file_names[i]))

    channel_timestamps = timestamps_of_all_senders[i]
    channel_message_sizes = message_sizes_of_all_senders[i]


    user_packet_timestamps = [p.get('timestamp') for p in packets_towards_receivers[i]]
    user_packet_sizes = [p.get('size') for p in packets_towards_receivers[i]]

    if len(user_packet_timestamps) == 0:
        all_user_bursts.append([])
        all_channel_bursts.append([])
        print('no receiver timestamps')
        continue
    start_time = min(channel_timestamps[0], user_packet_timestamps[0])
    print(channel_timestamps[0])
    print(user_packet_timestamps[0])
    print('start time: {}'.format(start_time))

    converted_sender_timestamps = convert_time(channel_timestamps, start_time)
    sender_period_points = get_on_periods_of_sender_2(converted_sender_timestamps, channel_message_sizes)

    converted_user_timestamps = convert_time(user_packet_timestamps, start_time)
    user_period_points = get_bursts_user(converted_user_timestamps, user_packet_sizes)

    all_user_bursts.append(user_period_points)
    all_channel_bursts.append(sender_period_points)
    print ('Hour {} is done'.format(i))
    print ("\n")

In [41]:
with open(os.path.join(results_dir, 'all_user_bursts.pickle'), 'wb') as handle:
    pickle.dump(all_user_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(results_dir, 'all_channel_bursts.pickle'), 'wb') as handle:
    pickle.dump(all_channel_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
with open(os.path.join(results_dir, 'all_user_bursts.pickle'), 'rb') as handle:
    all_user_bursts = pickle.load(handle)
with open(os.path.join(results_dir, 'all_channel_bursts.pickle'), 'rb') as handle:
    all_channel_bursts = pickle.load(handle)

Implement the technique

In [44]:
DELTA = 15 # seconds
size_threshold = 10000

In [45]:
def find_intervals(period_points, minute):
    global LENGTH
    num_of_intervals = int(LENGTH / minute)
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([])
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint:
                intervals[-1].append(period_points[j])
    return intervals

def find_intervals_channel(period_points, minute, message_types):
    global LENGTH
    num_of_intervals = int(LENGTH / minute)
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([])
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint and message_types[j] != 'text':
                intervals[-1].append(period_points[j])
    return intervals

def find_matches(channel_interval, user_interval):
    number_of_matches = 0
    number_of_nonmatches = 0
    matched_user_intervals = set()
    for j in range(min(len(channel_interval), len(user_interval))):
        time = channel_interval[j][0]
        size = channel_interval[j][1]
        is_matched = False
        for d in range(1, DELTA):
            for i in range(len(user_interval)):
                if user_interval[i] in matched_user_intervals:
                    continue
                if abs(user_interval[i][0] - time) < d:
                    if abs(size - user_interval[i][1]) < GAMMA * size:
                        matched_user_intervals.add(user_interval[i])
                        number_of_matches += 1
                        is_matched = True
                        break
            else:
                continue
            break
        if is_matched is False:
            number_of_nonmatches += 1
    return number_of_matches / float(number_of_matches + number_of_nonmatches)

def detect_correlations_from_bursts(channel_bursts, user_bursts, message_types, j):
    global observation_lengths
    correlations = []
    user_intervals = find_intervals(user_bursts, observation_lengths[j])
    channel_intervals = find_intervals_channel(channel_bursts, observation_lengths[j], message_types)
    correlation_of_intervals = detect_correlations_from_intervals(channel_intervals, user_intervals)
    return correlation_of_intervals

def detect_correlations_from_intervals(channel_intervals, user_intervals):
    correlation = []
    for i in range(len(channel_intervals)):
        if len(channel_intervals[i]) == 0 and len(user_intervals[i]) == 0: # no events in either one
            correlation.append(-1)
        elif len(channel_intervals[i]) == 0 or len(user_intervals[i]) == 0: # only one has an event
            correlation.append(0)
        else:
            correlation.append(find_matches(channel_intervals[i], user_intervals[i]))
    return correlation

def remove_empty_correlations(correlations):
    organized = []
    for i in range(len(correlations)):
        organized.append([c for c in correlations[i] if c != -1])
    return organized

### calculate correlations for positive samples (correlated pairs of flows)

In [ ]:
all_correlations_tp = []
for j in range(len(observation_lengths)):
    print('For {}-second interval:'.format(observation_lengths[j]))
    corrs_tp = []
    for i in range(len(all_user_bursts)): # for every hour
        corrs = detect_correlations_from_bursts(all_channel_bursts[i], all_user_bursts[i], message_types_of_all_senders[i], j)
        if corrs != -1:
            corrs_tp.extend(corrs)
        print ("Hour {} is done".format(i))
    all_correlations_tp.append([c for c in corrs_tp if c != -1])
with open(os.path.join(results_dir, 'corrs_tp_event_based_delta_{}.pickle'.format(DELTA)), 'wb') as handle:
    pickle.dump(all_correlations_tp, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Calculate correlations for negative samples (uncorrelated pairs of flows):

In [ ]:
# calculate correlations for false samples (uncorrelated pairs of flows)

all_corrs_fp = []
for m in range(len(observation_lengths)):
    print('For {}-second interval:'.format(observation_lengths[m]))
    
    corrs_fp = []
#     for j in range(len(all_user_bursts[i])):
    for i in range(len(all_user_bursts)):
        for j in range(len(all_channel_bursts)):
            if i == j:
                continue
            corrs = detect_correlations_from_bursts(all_channel_bursts[j], all_user_bursts[i], message_types_of_all_senders[j], m)
            if len(corrs) == 0:
                print('warning')
    #             continue
#             organized_correlations = remove_empty_correlations(corrs)
    #         for k in range(len(organized_correlations)):
    #             corrs_fp[k].extend(organized_correlations[k])
            if corrs != -1:
                corrs_fp.extend(corrs)
            print ("user {} with channel {} is done".format(i, j))
    all_corrs_fp.append([c for c in corrs_fp if c != -1])
with open(os.path.join(results_dir, 'corrs_fp_event_based_delta_{}.pickle'.format(DELTA)), 'wb') as handle:
    pickle.dump(all_corrs_fp, handle, protocol=pickle.HIGHEST_PROTOCOL)